# Caderno de Testes - Sistema Unificado Inove AI Framework

Testa a compatibilidade e funcionamento do sistema unificado para os 3 modelos:
- **Claude Code** (via `CLAUDE.md`)
- **Codex CLI** (via `AGENTS.md`)
- **Antigravity/Gemini** (via `GEMINI.md`)

---

In [ ]:
# Setup: ajusta o path para a raiz do projeto
import os
import sys
import json
import tempfile
import shutil
from pathlib import Path
from datetime import datetime, timedelta
from unittest.mock import patch

# Navega para a raiz do projeto
PROJECT_ROOT = Path(os.path.abspath('')).parent
if PROJECT_ROOT.name == 'inove-ai-framework':
    os.chdir(PROJECT_ROOT)
elif Path.cwd().name == 'tests':
    os.chdir(Path.cwd().parent)

PROJECT_ROOT = Path.cwd()
print(f"Diretorio do projeto: {PROJECT_ROOT}")

# Adiciona scripts ao path
sys.path.insert(0, str(PROJECT_ROOT / '.agents' / 'scripts'))

# Contadores de teste
PASSED = 0
FAILED = 0
SKIPPED = 0

def assert_test(condition, name, detail=""):
    """Helper para assercoes de teste."""
    global PASSED, FAILED
    if condition:
        PASSED += 1
        print(f"  PASS: {name}")
    else:
        FAILED += 1
        msg = f"  FAIL: {name}"
        if detail:
            msg += f" ({detail})"
        print(msg)

def skip_test(name, reason):
    """Marca teste como pulado."""
    global SKIPPED
    SKIPPED += 1
    print(f"  SKIP: {name} - {reason}")

def section(title):
    """Imprime cabecalho de secao."""
    print(f"\n{'='*60}")
    print(f" {title}")
    print(f"{'='*60}")

print("Setup concluido.")

## 1. Estrutura de Arquivos Bridge

Verifica se os arquivos bridge (CLAUDE.md, AGENTS.md, GEMINI.md) existem e apontam para o sistema canonico `.agents/`.

In [ ]:
section("1. ARQUIVOS BRIDGE")

# 1.1 Existencia dos bridges
bridge_files = {
    "CLAUDE.md": "Claude Code",
    "AGENTS.md": "Codex CLI",
    "GEMINI.md": "Antigravity/Gemini",
}

for filename, platform in bridge_files.items():
    filepath = PROJECT_ROOT / filename
    assert_test(filepath.exists(), f"{filename} existe ({platform})")

# 1.2 Bridges referenciam .agents/
for filename, platform in bridge_files.items():
    filepath = PROJECT_ROOT / filename
    if filepath.exists():
        content = filepath.read_text(encoding='utf-8')
        has_ref = '.agents/' in content or '.agents\\' in content
        assert_test(has_ref, f"{filename} referencia .agents/ ({platform})")
    else:
        skip_test(f"{filename} referencia .agents/", "arquivo nao encontrado")

# 1.3 Instrucoes canonicas existem
instructions = PROJECT_ROOT / '.agents' / 'INSTRUCTIONS.md'
assert_test(instructions.exists(), ".agents/INSTRUCTIONS.md existe (fonte canonica)")

# 1.4 Arquitetura documentada
architecture = PROJECT_ROOT / '.agents' / 'ARCHITECTURE.md'
assert_test(architecture.exists(), ".agents/ARCHITECTURE.md existe")

## 2. Estrutura do Diretorio `.agents/`

Valida que todos os subdiretorios obrigatorios existem e contem arquivos.

In [ ]:
section("2. ESTRUTURA .agents/")

agents_root = PROJECT_ROOT / '.agents'

# 2.1 Subdiretorios obrigatorios
required_dirs = ['agents', 'skills', 'workflows', 'scripts', 'config']
for d in required_dirs:
    dirpath = agents_root / d
    assert_test(dirpath.exists() and dirpath.is_dir(), f".agents/{d}/ existe")

# 2.2 Quantidade minima de agentes (esperado: 20)
agents_dir = agents_root / 'agents'
if agents_dir.exists():
    agent_files = list(agents_dir.glob('*.md'))
    assert_test(len(agent_files) >= 15, f"Agentes suficientes: {len(agent_files)} encontrados (min 15)")
    print(f"  INFO: Agentes encontrados: {[f.stem for f in sorted(agent_files)]}")
else:
    skip_test("Contagem de agentes", ".agents/agents/ nao encontrado")

# 2.3 Quantidade minima de skills (esperado: 36)
skills_dir = agents_root / 'skills'
if skills_dir.exists():
    skill_dirs = [d for d in skills_dir.iterdir() if d.is_dir()]
    skill_files = list(skills_dir.glob('*.md'))  # skills avulsas
    total_skills = len(skill_dirs) + len(skill_files)
    assert_test(total_skills >= 20, f"Skills suficientes: {total_skills} encontradas (min 20)")
else:
    skip_test("Contagem de skills", ".agents/skills/ nao encontrado")

# 2.4 Quantidade minima de workflows (esperado: 18)
workflows_dir = agents_root / 'workflows'
if workflows_dir.exists():
    workflow_files = list(workflows_dir.glob('*.md'))
    assert_test(len(workflow_files) >= 10, f"Workflows suficientes: {len(workflow_files)} encontrados (min 10)")
else:
    skip_test("Contagem de workflows", ".agents/workflows/ nao encontrado")

# 2.5 Scripts obrigatorios
required_scripts = [
    'platform_compat.py',
    'lock_manager.py',
    'auto_session.py',
    'finish_task.py',
    'progress_tracker.py',
    'dashboard.py',
    'sync_tracker.py',
    'session_logger.py',
    'metrics.py',
]

scripts_dir = agents_root / 'scripts'
for script in required_scripts:
    script_path = scripts_dir / script
    assert_test(script_path.exists(), f"Script {script} existe")

## 3. Symlinks do Codex CLI (`.codex/`)

Valida que os symlinks do Codex apontam corretamente para o sistema canonico.

In [ ]:
section("3. CODEX CLI - SYMLINKS")

codex_dir = PROJECT_ROOT / '.codex'

# 3.1 Diretorio .codex existe
assert_test(codex_dir.exists(), ".codex/ diretorio existe")

if codex_dir.exists():
    # 3.2 config.toml existe
    codex_config = codex_dir / 'config.toml'
    assert_test(codex_config.exists(), ".codex/config.toml existe")

    # 3.3 Symlink de skills
    skills_link = codex_dir / 'skills'
    if skills_link.exists():
        is_symlink = skills_link.is_symlink()
        assert_test(is_symlink, ".codex/skills e um symlink")
        if is_symlink:
            target = os.readlink(skills_link)
            assert_test(
                '.agents/skills' in target or '../.agents/skills' in target,
                f".codex/skills aponta para .agents/skills (target: {target})"
            )
            # Verifica se o symlink resolve corretamente
            resolved = skills_link.resolve()
            assert_test(resolved.exists(), ".codex/skills resolve para diretorio existente")
    else:
        skip_test(".codex/skills symlink", "nao encontrado")

    # 3.4 Symlink de prompts/workflows
    prompts_link = codex_dir / 'prompts'
    if prompts_link.exists():
        is_symlink = prompts_link.is_symlink()
        assert_test(is_symlink, ".codex/prompts e um symlink")
        if is_symlink:
            target = os.readlink(prompts_link)
            assert_test(
                '.agents/workflows' in target or '../.agents/workflows' in target,
                f".codex/prompts aponta para .agents/workflows (target: {target})"
            )
            resolved = prompts_link.resolve()
            assert_test(resolved.exists(), ".codex/prompts resolve para diretorio existente")
    else:
        skip_test(".codex/prompts symlink", "nao encontrado")

    # 3.5 Conteudo acessivel via symlink == conteudo original
    if skills_link.exists() and skills_link.is_symlink():
        original_skills = set(f.name for f in (agents_root / 'skills').iterdir())
        linked_skills = set(f.name for f in skills_link.iterdir())
        assert_test(
            original_skills == linked_skills,
            f"Skills via symlink identicas ao original ({len(original_skills)} items)"
        )
else:
    skip_test("Todos os testes Codex", ".codex/ nao encontrado")

## 4. Platform Compat - Deteccao de Plataforma

Testa o modulo `platform_compat.py` simulando cada plataforma via variaveis de ambiente.

In [ ]:
section("4. PLATFORM_COMPAT - DETECCAO DE PLATAFORMA")

# Reimporta para garantir estado limpo
import importlib
import platform_compat
importlib.reload(platform_compat)

# 4.1 get_agent_root() retorna Path valido
root = platform_compat.get_agent_root()
assert_test(isinstance(root, Path), f"get_agent_root() retorna Path: {root}")
assert_test(root.exists(), f"get_agent_root() aponta para diretorio existente")

# 4.2 Paths auxiliares
assert_test(platform_compat.get_skills_path().exists(), "get_skills_path() valido")
assert_test(platform_compat.get_agents_path().exists(), "get_agents_path() valido")
assert_test(platform_compat.get_workflows_path().exists(), "get_workflows_path() valido")
assert_test(platform_compat.get_scripts_path().exists(), "get_scripts_path() valido")

# 4.3 Simular Claude Code
env_clean = {k: v for k, v in os.environ.items()
             if k not in ('CLAUDE_CODE_SESSION', 'CODEX_SESSION',
                         'ANTIGRAVITY_SESSION', 'GEMINI_SESSION', 'AGENT_SOURCE')}

with patch.dict(os.environ, {**env_clean, 'CLAUDE_CODE_SESSION': '1'}, clear=True):
    importlib.reload(platform_compat)
    result = platform_compat.get_agent_source()
    assert_test(result == 'claude_code', f"Claude Code detectado: {result}")

# 4.4 Simular Codex CLI
with patch.dict(os.environ, {**env_clean, 'CODEX_SESSION': '1'}, clear=True):
    importlib.reload(platform_compat)
    result = platform_compat.get_agent_source()
    assert_test(result == 'codex', f"Codex CLI detectado: {result}")

# 4.5 Simular Antigravity via ANTIGRAVITY_SESSION
with patch.dict(os.environ, {**env_clean, 'ANTIGRAVITY_SESSION': '1'}, clear=True):
    importlib.reload(platform_compat)
    result = platform_compat.get_agent_source()
    assert_test(result == 'antigravity', f"Antigravity (ANTIGRAVITY_SESSION) detectado: {result}")

# 4.6 Simular Antigravity via GEMINI_SESSION
with patch.dict(os.environ, {**env_clean, 'GEMINI_SESSION': '1'}, clear=True):
    importlib.reload(platform_compat)
    result = platform_compat.get_agent_source()
    assert_test(result == 'antigravity', f"Antigravity (GEMINI_SESSION) detectado: {result}")

# 4.7 Simular AGENT_SOURCE explicito
with patch.dict(os.environ, {**env_clean, 'AGENT_SOURCE': 'codex'}, clear=True):
    importlib.reload(platform_compat)
    result = platform_compat.get_agent_source()
    assert_test(result == 'codex', f"AGENT_SOURCE=codex fallback detectado: {result}")

# 4.8 Sem variavel de ambiente -> 'unknown'
with patch.dict(os.environ, env_clean, clear=True):
    importlib.reload(platform_compat)
    result = platform_compat.get_agent_source()
    assert_test(result == 'unknown', f"Sem env vars -> unknown: {result}")

# 4.9 Prioridade: CODEX_SESSION > AGENT_SOURCE
with patch.dict(os.environ, {**env_clean, 'CODEX_SESSION': '1', 'AGENT_SOURCE': 'antigravity'}, clear=True):
    importlib.reload(platform_compat)
    result = platform_compat.get_agent_source()
    assert_test(result == 'codex', f"Prioridade CODEX_SESSION > AGENT_SOURCE: {result}")

# 4.10 get_config_path() retorna path correto por plataforma
importlib.reload(platform_compat)
config_codex = platform_compat.get_config_path('codex')
assert_test('codex.toml' in str(config_codex), f"Config codex: {config_codex}")

config_claude = platform_compat.get_config_path('claude_code')
assert_test('claude.json' in str(config_claude), f"Config claude: {config_claude}")

config_anti = platform_compat.get_config_path('antigravity')
assert_test('antigravity.json' in str(config_anti), f"Config antigravity: {config_anti}")

## 5. Lock Manager - Coordenacao Multi-Agent

Testa acquire/release de locks, deteccao de stale locks, e conflitos entre agentes.

In [ ]:
section("5. LOCK MANAGER")

from lock_manager import LockManager

# Usa diretorio temporario para nao afetar locks reais
test_locks_dir = Path(tempfile.mkdtemp(prefix='inove_test_locks_'))
lm = LockManager(locks_dir=test_locks_dir, default_timeout=5)

try:
    # 5.1 Acquire lock
    result = lm.acquire_lock('backlog', 'claude_code')
    assert_test(result == True, "acquire_lock sucesso")

    # 5.2 Lock info disponivel
    info = lm.get_lock_info('backlog')
    assert_test(info is not None, "get_lock_info retorna dados")
    assert_test(info['locked_by'] == 'claude_code', f"Lock pertence a claude_code: {info.get('locked_by')}")

    # 5.3 Mesmo agente pode renovar lock
    result = lm.acquire_lock('backlog', 'claude_code')
    assert_test(result == True, "Mesmo agente renova lock")

    # 5.4 Outro agente NAO pode adquirir lock ativo
    result = lm.acquire_lock('backlog', 'antigravity')
    assert_test(result == False, "Agente diferente bloqueado")

    # 5.5 Release pelo dono
    result = lm.release_lock('backlog', 'claude_code')
    assert_test(result == True, "Release pelo dono sucesso")

    # 5.6 Apos release, outro agente pode adquirir
    result = lm.acquire_lock('backlog', 'antigravity')
    assert_test(result == True, "Antigravity adquire apos release")

    # 5.7 Release por agente errado falha
    result = lm.release_lock('backlog', 'claude_code')
    assert_test(result == False, "Release por agente errado falha")

    # 5.8 Force release funciona
    result = lm.force_release('backlog')
    assert_test(result == True, "Force release sucesso")
    info = lm.get_lock_info('backlog')
    assert_test(info is None, "Lock removido apos force release")

    # 5.9 Multiplos locks independentes
    lm.acquire_lock('backlog', 'claude_code')
    lm.acquire_lock('readme', 'antigravity')
    lm.acquire_lock('config', 'codex')
    active = lm.list_active_locks()
    assert_test(len(active) == 3, f"3 locks ativos simultaneos: {len(active)}")
    assert_test(active['backlog']['locked_by'] == 'claude_code', "Backlog -> claude_code")
    assert_test(active['readme']['locked_by'] == 'antigravity', "Readme -> antigravity")
    assert_test(active['config']['locked_by'] == 'codex', "Config -> codex")

    # 5.10 Cleanup stale locks (timeout=5s, entao os locks ja devem estar validos)
    count = lm.cleanup_stale_locks()
    assert_test(count == 0, f"Nenhum lock stale (timeout=5s): {count} removidos")

    # Limpa para o proximo teste
    for r in ['backlog', 'readme', 'config']:
        lm.force_release(r)

    # 5.11 Deteccao de stale lock (simula lock expirado)
    lock_file = test_locks_dir / 'stale_resource.lock'
    stale_data = {
        'locked_by': 'antigravity',
        'locked_at': (datetime.now() - timedelta(seconds=600)).isoformat(),
        'timeout': 5,
        'pid': 99999
    }
    lock_file.write_text(json.dumps(stale_data))
    info = lm.get_lock_info('stale_resource')
    assert_test(info is None, "Lock stale detectado e removido automaticamente")

    # 5.12 Lock com metadata extra
    result = lm.acquire_lock('feature-x', 'claude_code', task='Epic-1', description='Auth')
    assert_test(result == True, "Lock com metadata adquirido")
    info = lm.get_lock_info('feature-x')
    assert_test(info.get('task') == 'Epic-1', f"Metadata 'task' preservada: {info.get('task')}")
    assert_test(info.get('description') == 'Auth', f"Metadata 'description' preservada")
    lm.force_release('feature-x')

    # 5.13 Release de recurso inexistente retorna True
    result = lm.release_lock('nao_existe', 'claude_code')
    assert_test(result == True, "Release de recurso inexistente retorna True")

finally:
    # Cleanup
    shutil.rmtree(test_locks_dir, ignore_errors=True)
    print(f"  INFO: Diretorio temporario limpo: {test_locks_dir}")

## 6. Auto Session - Gerenciamento de Sessao

Testa inicio, status e encerramento de sessoes para cada plataforma.

In [ ]:
section("6. AUTO SESSION")

import auto_session
importlib.reload(auto_session)

# Salva estado original da sessao se existir
original_session_file = Path('.agents/.session_state.json')
original_session_backup = None
if original_session_file.exists():
    original_session_backup = original_session_file.read_text()

try:
    # Limpa sessao existente para testes
    auto_session.clear_session()

    # 6.1 Deteccao de agente - Claude Code
    with patch.dict(os.environ, {'CLAUDE_CODE_SESSION': '1'}, clear=False):
        result = auto_session.get_agent_source()
        assert_test(result == 'claude_code', f"Session detecta Claude Code: {result}")

    # 6.2 Deteccao de agente - Antigravity
    env_clean_session = {k: v for k, v in os.environ.items()
                        if k not in ('CLAUDE_CODE_SESSION', 'CODEX_SESSION',
                                    'ANTIGRAVITY_SESSION', 'GEMINI_SESSION', 'AGENT_SOURCE')}
    with patch.dict(os.environ, {**env_clean_session, 'GEMINI_SESSION': '1'}, clear=True):
        result = auto_session.get_agent_source()
        assert_test(result == 'antigravity', f"Session detecta Antigravity: {result}")

    # 6.3 Iniciar sessao com override
    auto_session.clear_session()
    result = auto_session.start_session(agent_override='claude_code')
    assert_test(result == True, "Sessao iniciada com sucesso")

    # 6.4 Sessao salva corretamente
    session = auto_session.load_session()
    assert_test(session is not None, "Sessao carregada do arquivo")
    assert_test(session['agent'] == 'claude_code', f"Agente da sessao: {session.get('agent')}")
    assert_test(session['ended'] == False, "Sessao nao encerrada")
    assert_test('start_time' in session, "start_time presente")
    assert_test('date' in session, "date presente")
    assert_test('project' in session, f"project presente: {session.get('project')}")

    # 6.5 Nao pode iniciar sessao duplicada
    result = auto_session.start_session(agent_override='antigravity')
    assert_test(result == False, "Sessao duplicada bloqueada")

    # 6.6 Encerrar sessao
    result = auto_session.end_session(activities='Teste unitario; Validacao do sistema')
    assert_test(result == True, "Sessao encerrada com sucesso")

    # 6.7 Nao pode encerrar sessao ja encerrada
    result = auto_session.end_session()
    assert_test(result == False, "Encerrar sessao ja encerrada falha")

    # 6.8 Iniciar sessao como Antigravity
    result = auto_session.start_session(agent_override='antigravity')
    assert_test(result == True, "Sessao Antigravity iniciada")
    session = auto_session.load_session()
    assert_test(session['agent'] == 'antigravity', "Agente Antigravity correto")

    # 6.9 Encerrar e verificar limpeza
    auto_session.end_session(quick=True)
    session = auto_session.load_session()
    assert_test(session is None, "Sessao limpa apos encerramento")

    # 6.10 Iniciar sessao como Codex
    result = auto_session.start_session(agent_override='codex')
    assert_test(result == True, "Sessao Codex iniciada")
    session = auto_session.load_session()
    assert_test(session['agent'] == 'codex', "Agente Codex correto")
    auto_session.end_session(quick=True)

finally:
    # Restaura sessao original
    auto_session.clear_session()
    if original_session_backup:
        original_session_file.write_text(original_session_backup)
        print("  INFO: Sessao original restaurada")
    else:
        print("  INFO: Nenhuma sessao original para restaurar")

## 7. Configuracoes por Plataforma

Valida que os arquivos de configuracao existem e tem formato correto.

In [ ]:
section("7. CONFIGURACOES POR PLATAFORMA")

config_dir = agents_root / 'config'

# 7.1 codex.toml existe e e parseavel
codex_toml = config_dir / 'codex.toml'
if codex_toml.exists():
    content = codex_toml.read_text()
    assert_test(len(content) > 0, "codex.toml tem conteudo")
    # Verifica campos basicos
    assert_test('model' in content, "codex.toml contem 'model'")
    assert_test('approval_policy' in content or 'sandbox' in content,
                "codex.toml contem politicas de aprovacao ou sandbox")
else:
    skip_test("codex.toml", "arquivo nao encontrado")

# 7.2 mcp.json existe e e JSON valido
mcp_json = config_dir / 'mcp.json'
if mcp_json.exists():
    try:
        data = json.loads(mcp_json.read_text())
        assert_test(isinstance(data, dict), "mcp.json e JSON valido")
    except json.JSONDecodeError as e:
        assert_test(False, f"mcp.json JSON invalido: {e}")
else:
    skip_test("mcp.json", "arquivo nao encontrado")

# 7.3 Claude settings
claude_settings = PROJECT_ROOT / '.claude' / 'settings.json'
if claude_settings.exists():
    try:
        data = json.loads(claude_settings.read_text())
        assert_test(isinstance(data, dict), ".claude/settings.json e JSON valido")
    except json.JSONDecodeError as e:
        assert_test(False, f".claude/settings.json JSON invalido: {e}")
else:
    skip_test(".claude/settings.json", "arquivo nao encontrado")

# 7.4 Gemini rules
gemini_rules = agents_root / 'rules' / 'GEMINI.md'
if gemini_rules.exists():
    content = gemini_rules.read_text()
    assert_test(len(content) > 100, f"GEMINI.md rules tem conteudo ({len(content)} chars)")
else:
    skip_test("GEMINI.md rules", "arquivo nao encontrado")

## 8. Consistencia de Agentes

Valida que todos os agentes tem a estrutura correta (frontmatter YAML + corpo markdown).

In [ ]:
section("8. CONSISTENCIA DE AGENTES")

import re

agents_dir = agents_root / 'agents'
agent_files = sorted(agents_dir.glob('*.md')) if agents_dir.exists() else []

expected_agents = [
    'orchestrator', 'frontend-specialist', 'backend-specialist',
    'database-architect', 'mobile-developer', 'security-auditor',
    'debugger', 'devops-engineer', 'test-engineer', 'product-owner',
    'qa-automation-engineer', 'documentation-writer', 'code-archaeologist',
    'performance-optimizer', 'explorer-agent', 'project-planner',
    'product-manager', 'seo-specialist', 'penetration-tester', 'game-developer',
]

# 8.1 Agentes esperados existem
existing_stems = [f.stem for f in agent_files]
for agent_name in expected_agents:
    assert_test(agent_name in existing_stems, f"Agente {agent_name} existe")

# 8.2 Cada agente tem frontmatter YAML valido
frontmatter_pattern = re.compile(r'^---\s*\n(.+?)\n---', re.DOTALL)

for agent_file in agent_files:
    content = agent_file.read_text(encoding='utf-8')
    match = frontmatter_pattern.match(content)
    has_fm = match is not None
    assert_test(has_fm, f"{agent_file.stem}: tem frontmatter YAML")

    if has_fm:
        fm_text = match.group(1)
        # Verifica campos obrigatorios
        has_name = 'name:' in fm_text
        has_desc = 'description:' in fm_text
        if not has_name:
            assert_test(False, f"{agent_file.stem}: falta 'name' no frontmatter")
        if not has_desc:
            assert_test(False, f"{agent_file.stem}: falta 'description' no frontmatter")

## 9. Consistencia de Skills

Verifica que skills referenciadas pelos agentes existem no diretorio de skills.

In [ ]:
section("9. CONSISTENCIA DE SKILLS")

skills_dir = agents_root / 'skills'

# 9.1 Coletar todas as skills referenciadas nos agentes
referenced_skills = set()
for agent_file in agent_files:
    content = agent_file.read_text(encoding='utf-8')
    match = frontmatter_pattern.match(content)
    if match:
        fm_text = match.group(1)
        skills_match = re.search(r'skills:\s*(.+)', fm_text)
        if skills_match:
            skills_str = skills_match.group(1).strip()
            # Pode ser lista YAML ou CSV
            if skills_str.startswith('['):
                skills_list = [s.strip().strip('"').strip("'") for s in skills_str.strip('[]').split(',')]
            else:
                skills_list = [s.strip() for s in skills_str.split(',')]
            referenced_skills.update(s for s in skills_list if s)

print(f"  INFO: {len(referenced_skills)} skills referenciadas pelos agentes")

# 9.2 Verificar que cada skill referenciada existe
available_skills = set()
if skills_dir.exists():
    for item in skills_dir.iterdir():
        if item.is_dir():
            available_skills.add(item.name)
        elif item.suffix == '.md':
            available_skills.add(item.stem)

print(f"  INFO: {len(available_skills)} skills disponiveis")

missing_skills = referenced_skills - available_skills
for skill in sorted(referenced_skills):
    if skill in available_skills:
        assert_test(True, f"Skill '{skill}' existe")
    else:
        assert_test(False, f"Skill '{skill}' referenciada mas NAO encontrada")

# 9.3 Skills com SKILL.md (index)
skill_dirs_with_index = 0
skill_dirs_total = 0
for item in skills_dir.iterdir():
    if item.is_dir() and not item.name.startswith('.'):
        skill_dirs_total += 1
        if (item / 'SKILL.md').exists():
            skill_dirs_with_index += 1

if skill_dirs_total > 0:
    pct = skill_dirs_with_index / skill_dirs_total * 100
    assert_test(pct >= 80, f"{skill_dirs_with_index}/{skill_dirs_total} skill dirs tem SKILL.md ({pct:.0f}%)")

## 10. Workflows Cross-Platform

Verifica que os workflows sao acessiveis por todas as plataformas.

In [ ]:
section("10. WORKFLOWS CROSS-PLATFORM")

workflows_dir = agents_root / 'workflows'

expected_workflows = [
    'define', 'brainstorm', 'create', 'debug', 'enhance',
    'deploy', 'test', 'track', 'status', 'finish',
    'context', 'readiness', 'journeys', 'log',
]

# 10.1 Workflows esperados existem
if workflows_dir.exists():
    existing_workflows = [f.stem for f in workflows_dir.glob('*.md')]
    for wf_name in expected_workflows:
        assert_test(wf_name in existing_workflows, f"Workflow '{wf_name}' existe")
else:
    skip_test("Workflows", "diretorio nao encontrado")

# 10.2 Workflows acessiveis via .codex/prompts (symlink)
codex_prompts = PROJECT_ROOT / '.codex' / 'prompts'
if codex_prompts.exists():
    codex_workflows = [f.stem for f in codex_prompts.glob('*.md')]
    for wf_name in expected_workflows:
        assert_test(
            wf_name in codex_workflows,
            f"Workflow '{wf_name}' acessivel via .codex/prompts"
        )
else:
    skip_test("Workflows via Codex", ".codex/prompts nao encontrado")

# 10.3 Conteudo identico entre acesso direto e symlink
if codex_prompts.exists() and workflows_dir.exists():
    sample_wf = 'define.md'
    direct = (workflows_dir / sample_wf)
    via_symlink = (codex_prompts / sample_wf)
    if direct.exists() and via_symlink.exists():
        assert_test(
            direct.read_text() == via_symlink.read_text(),
            f"Conteudo de '{sample_wf}' identico via acesso direto e symlink"
        )

## 11. Deteccao de Agente nos Scripts

Verifica que todos os scripts que usam deteccao de agente sao consistentes.

In [ ]:
section("11. DETECCAO DE AGENTE CONSISTENTE")

scripts_dir = agents_root / 'scripts'
scripts_with_detection = []

for script_file in sorted(scripts_dir.glob('*.py')):
    content = script_file.read_text(encoding='utf-8')
    # Procura por funcoes de deteccao de agente
    has_detection = any([
        'get_agent_source' in content,
        'AGENT_SOURCE' in content,
        'CLAUDE_CODE_SESSION' in content,
        'GEMINI_SESSION' in content,
        'CODEX_SESSION' in content,
    ])
    if has_detection:
        scripts_with_detection.append(script_file.name)

print(f"  INFO: {len(scripts_with_detection)} scripts com deteccao de agente")

# 11.1 Scripts criticos tem deteccao
critical_scripts = ['lock_manager.py', 'auto_session.py', 'finish_task.py', 'platform_compat.py']
for script in critical_scripts:
    assert_test(
        script in scripts_with_detection,
        f"{script} tem deteccao de agente"
    )

# 11.2 Todos os scripts que detectam agente suportam os 3 modelos
for script_name in scripts_with_detection:
    content = (scripts_dir / script_name).read_text(encoding='utf-8')
    supports_claude = 'claude_code' in content or 'CLAUDE_CODE_SESSION' in content
    supports_gemini = 'antigravity' in content or 'GEMINI_SESSION' in content or 'ANTIGRAVITY_SESSION' in content
    
    # Pelo menos Claude e Antigravity devem ser suportados (Codex pode usar AGENT_SOURCE)
    if supports_claude and supports_gemini:
        assert_test(True, f"{script_name}: suporta Claude + Antigravity")
    else:
        missing = []
        if not supports_claude:
            missing.append('Claude Code')
        if not supports_gemini:
            missing.append('Antigravity')
        assert_test(False, f"{script_name}: falta suporte para {', '.join(missing)}")

## 12. Integracao End-to-End: Fluxo Completo por Plataforma

Simula o fluxo completo de um agente em cada plataforma: deteccao -> sessao -> lock -> release.

In [ ]:
section("12. INTEGRACAO E2E - FLUXO COMPLETO")

test_locks_dir_e2e = Path(tempfile.mkdtemp(prefix='inove_e2e_locks_'))
original_session_backup_e2e = None
if original_session_file.exists():
    original_session_backup_e2e = original_session_file.read_text()

platforms = [
    ('claude_code', {'CLAUDE_CODE_SESSION': '1'}),
    ('codex', {'CODEX_SESSION': '1'}),
    ('antigravity', {'GEMINI_SESSION': '1'}),
]

try:
    for platform_name, env_vars in platforms:
        print(f"\n  --- Testando plataforma: {platform_name} ---")

        # Limpa estado
        auto_session.clear_session()
        lm_e2e = LockManager(locks_dir=test_locks_dir_e2e, default_timeout=60)
        for f in test_locks_dir_e2e.glob('*.lock'):
            f.unlink()

        env_base = {k: v for k, v in os.environ.items()
                    if k not in ('CLAUDE_CODE_SESSION', 'CODEX_SESSION',
                                'ANTIGRAVITY_SESSION', 'GEMINI_SESSION', 'AGENT_SOURCE')}

        with patch.dict(os.environ, {**env_base, **env_vars}, clear=True):
            # Step 1: Deteccao de plataforma
            importlib.reload(platform_compat)
            detected = platform_compat.get_agent_source()
            assert_test(
                detected == platform_name,
                f"[{platform_name}] Plataforma detectada: {detected}"
            )

            # Step 2: Paths resolvem corretamente
            root = platform_compat.get_agent_root()
            assert_test(root.exists(), f"[{platform_name}] Agent root existe")

            # Step 3: Iniciar sessao
            result = auto_session.start_session(agent_override=platform_name)
            assert_test(result, f"[{platform_name}] Sessao iniciada")

            # Step 4: Adquirir lock
            result = lm_e2e.acquire_lock('backlog', platform_name)
            assert_test(result, f"[{platform_name}] Lock adquirido")

            # Step 5: Verificar lock info
            info = lm_e2e.get_lock_info('backlog')
            assert_test(
                info and info['locked_by'] == platform_name,
                f"[{platform_name}] Lock pertence ao agente correto"
            )

            # Step 6: Liberar lock
            result = lm_e2e.release_lock('backlog', platform_name)
            assert_test(result, f"[{platform_name}] Lock liberado")

            # Step 7: Encerrar sessao
            result = auto_session.end_session(quick=True)
            assert_test(result, f"[{platform_name}] Sessao encerrada")

    # Teste de conflito entre plataformas
    print(f"\n  --- Teste de conflito multi-agent ---")
    for f in test_locks_dir_e2e.glob('*.lock'):
        f.unlink()

    lm_conflict = LockManager(locks_dir=test_locks_dir_e2e, default_timeout=60)

    # Claude Code adquire lock
    lm_conflict.acquire_lock('shared_resource', 'claude_code')

    # Codex tenta adquirir o mesmo recurso
    result = lm_conflict.acquire_lock('shared_resource', 'codex')
    assert_test(result == False, "Conflito: Codex bloqueado por Claude Code")

    # Antigravity tenta adquirir o mesmo recurso
    result = lm_conflict.acquire_lock('shared_resource', 'antigravity')
    assert_test(result == False, "Conflito: Antigravity bloqueado por Claude Code")

    # Claude Code libera
    lm_conflict.release_lock('shared_resource', 'claude_code')

    # Agora Codex consegue
    result = lm_conflict.acquire_lock('shared_resource', 'codex')
    assert_test(result == True, "Apos release: Codex adquire lock")

    # Antigravity ainda bloqueado
    result = lm_conflict.acquire_lock('shared_resource', 'antigravity')
    assert_test(result == False, "Conflito: Antigravity bloqueado por Codex")

    lm_conflict.force_release('shared_resource')

finally:
    shutil.rmtree(test_locks_dir_e2e, ignore_errors=True)
    auto_session.clear_session()
    if original_session_backup_e2e:
        original_session_file.write_text(original_session_backup_e2e)

## 13. Validacao de Migracao

Verifica que nao restam referencias ao sistema legado `.agent/` (sem 's').

In [ ]:
section("13. VALIDACAO DE MIGRACAO")

# 13.1 Diretorio legado .agent/ nao existe
legacy_dir = PROJECT_ROOT / '.agent'
assert_test(
    not legacy_dir.exists(),
    f"Diretorio legado .agent/ NAO existe (migrado para .agents/)"
)

# 13.2 Script de migracao existe
migrate_script = PROJECT_ROOT / 'migrate_to_unified.py'
assert_test(migrate_script.exists(), "migrate_to_unified.py existe")

# 13.3 Bridges NAO contem path legado hardcoded
for filename in ['CLAUDE.md', 'AGENTS.md', 'GEMINI.md']:
    filepath = PROJECT_ROOT / filename
    if filepath.exists():
        content = filepath.read_text(encoding='utf-8')
        # Procura por .agent/ (sem s) que NAO seja parte de .agents/
        # Regex: .agent/ nao seguido de 's'
        legacy_refs = re.findall(r'\.agent/(?!s)', content)
        assert_test(
            len(legacy_refs) == 0,
            f"{filename}: sem referencias legadas .agent/ ({len(legacy_refs)} encontradas)"
        )

## 13.5 Symlinks Nativos do Claude Code (`.claude/`)

Valida que os symlinks nativos do Claude Code apontam para o sistema canonico `.agents/`.

In [ ]:
section("13.5 SYMLINKS NATIVOS - CLAUDE CODE E CODEX")

# --- Claude Code (.claude/) ---
claude_dir = PROJECT_ROOT / '.claude'

# 13.5.1 .claude/agents e symlink
claude_agents = claude_dir / 'agents'
if claude_agents.exists():
    assert_test(claude_agents.is_symlink(), ".claude/agents e um symlink")
    if claude_agents.is_symlink():
        target = os.readlink(claude_agents)
        assert_test(
            '.agents/agents' in target,
            f".claude/agents aponta para .agents/agents (target: {target})"
        )
        assert_test(claude_agents.resolve().exists(), ".claude/agents resolve para diretorio existente")
else:
    skip_test(".claude/agents", "nao encontrado")

# 13.5.2 .claude/skills e symlink
claude_skills = claude_dir / 'skills'
if claude_skills.exists():
    assert_test(claude_skills.is_symlink(), ".claude/skills e um symlink")
    if claude_skills.is_symlink():
        target = os.readlink(claude_skills)
        assert_test(
            '.agents/skills' in target,
            f".claude/skills aponta para .agents/skills (target: {target})"
        )
        assert_test(claude_skills.resolve().exists(), ".claude/skills resolve para diretorio existente")
else:
    skip_test(".claude/skills", "nao encontrado")

# 13.5.3 Conteudo via .claude/agents == .agents/agents
if claude_agents.exists() and claude_agents.is_symlink():
    original = set(f.name for f in (agents_root / 'agents').iterdir() if f.suffix == '.md')
    via_link = set(f.name for f in claude_agents.iterdir() if f.suffix == '.md')
    assert_test(original == via_link, f".claude/agents: conteudo identico ({len(original)} agentes)")

# 13.5.4 Conteudo via .claude/skills == .agents/skills
if claude_skills.exists() and claude_skills.is_symlink():
    original = set(f.name for f in (agents_root / 'skills').iterdir())
    via_link = set(f.name for f in claude_skills.iterdir())
    assert_test(original == via_link, f".claude/skills: conteudo identico ({len(original)} skills)")

# --- Codex CLI (.codex/agents) ---
codex_agents = PROJECT_ROOT / '.codex' / 'agents'
if codex_agents.exists():
    assert_test(codex_agents.is_symlink(), ".codex/agents e um symlink")
    if codex_agents.is_symlink():
        target = os.readlink(codex_agents)
        assert_test(
            '.agents/agents' in target,
            f".codex/agents aponta para .agents/agents (target: {target})"
        )
        assert_test(codex_agents.resolve().exists(), ".codex/agents resolve para diretorio existente")

    # Conteudo identico
    original = set(f.name for f in (agents_root / 'agents').iterdir() if f.suffix == '.md')
    via_link = set(f.name for f in codex_agents.iterdir() if f.suffix == '.md')
    assert_test(original == via_link, f".codex/agents: conteudo identico ({len(original)} agentes)")
else:
    skip_test(".codex/agents", "nao encontrado")

# --- Cross-check: todos os 3 acessos retornam o mesmo conteudo ---
if all(p.exists() for p in [agents_root / 'agents', claude_agents, codex_agents]):
    direct = set(f.name for f in (agents_root / 'agents').iterdir() if f.suffix == '.md')
    via_claude = set(f.name for f in claude_agents.iterdir() if f.suffix == '.md')
    via_codex = set(f.name for f in codex_agents.iterdir() if f.suffix == '.md')
    assert_test(
        direct == via_claude == via_codex,
        f"Cross-check: 3 plataformas veem os mesmos {len(direct)} agentes"
    )

## 14. Resumo dos Testes

Exibe o resultado consolidado de todos os testes.

In [ ]:
section("RESUMO FINAL")

total = PASSED + FAILED + SKIPPED
pass_rate = (PASSED / total * 100) if total > 0 else 0

print(f"")
print(f"  Total de testes:  {total}")
print(f"  Passou:           {PASSED}")
print(f"  Falhou:           {FAILED}")
print(f"  Pulados:          {SKIPPED}")
print(f"  Taxa de sucesso:  {pass_rate:.1f}%")
print(f"")

# Barra visual
bar_width = 40
filled = int(bar_width * PASSED / total) if total > 0 else 0
bar = '#' * filled + '-' * (bar_width - filled)
print(f"  [{bar}] {pass_rate:.1f}%")
print(f"")

if FAILED == 0:
    print("  RESULTADO: TODOS OS TESTES PASSARAM!")
    print("  O sistema unificado esta funcionando corretamente para os 3 modelos.")
else:
    print(f"  RESULTADO: {FAILED} TESTE(S) FALHARAM")
    print(f"  Revise os itens marcados com FAIL acima.")

print(f"")
print(f"  Plataformas testadas:")
print(f"    - Claude Code  (CLAUDE.md -> .agents/)")
print(f"    - Codex CLI    (AGENTS.md -> .codex/ -> .agents/)")
print(f"    - Antigravity  (GEMINI.md -> .agents/)")